# How to build p-net
## Prepare environment and define parameters

In [ ]:
is_colab = True
import sys

if is_colab:
    !pip install -q torch==1.9.0
    !pip install -q torchvision==0.10.0
    !pip install -q qiskit==0.20.0
    !pip install qfnn
    !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1w9VRv0iVfsH20Kb_MkF3yFhFeiYDVy5n' -O model.tar.gz
    !tar zxvf /content/model.tar.gz

import torch

from qiskit import  QuantumCircuit, ClassicalRegister
import functools


from qfnn.qf_fb.q_output import fire_ibmq,analyze,add_measure
from qfnn.qf_circ.p_lyr_circ import P_LYR_Circ,P_Neuron_Circ
from qfnn.qf_fb.c_input import load_data,to_quantum_matrix

from qfnn.qf_net.p_lyr import P_LYR
import torch.nn as nn
print = functools.partial(print, flush=True)


## Inference
### Get parameters

At beginning, we need to extract the weights and related parameters
from the model. These parameters will be used to build the quantum circuit.

In [ ]:
# Model initialization
weight_1 = torch.tensor([[1.,  -1.,  1.,  1.],[-1., 1., 1., 1.]]) # weights of layer 2
weight_2 = torch.tensor([[1.,  -1.],[-1.,  -1.]]) # weights of layer 1
input = [0,0,0,0]   # input

### An example to implement a P-LYR based quantum neuron

In the following cell, we will use *P_Neuron_Circ* to create the circuit for the
quantum neuron designed based on P-LYR in [QuantumFlow](https://www.nature.com/articles/s41467-020-20729-5).

**Note**: we will use the weight *weight_1* for embedding the weights to the circuit.


In [ ]:
#create circuit
circuit_demo = QuantumCircuit()

#init circuit
p_layer_example = P_Neuron_Circ(4)

#create qubits to be invovled and store them
inps = p_layer_example.add_input_qubits(circuit_demo,'p_input')
aux =p_layer_example.add_aux(circuit_demo,'aux_qubit')
output = p_layer_example.add_out_qubits(circuit_demo,'p_out_qubit')

#add p-neuron to the circuit
p_layer_example.forward(circuit_demo,[weight_1[0]],inps[0],output,aux,input)

#show your circuit
circuit_demo.draw('text',fold=300)

### Build the QF-pNet network

In the following cell, we build the first layer of the QF-pNet using the qfnn library.

In [ ]:
input_list = []
aux_list = []
output_list = []
circuit = QuantumCircuit()
for i in range(2):
    #init circuit
    p_layer = P_Neuron_Circ(4)

    #create and store qubits
    inps = p_layer.add_input_qubits(circuit,'p'+str(i)+"_input")
    aux =p_layer.add_aux(circuit,'aux'+str(i)+"_qubit")
    output = p_layer.add_out_qubits(circuit,'p_out_'+str(i)+"_qubit")

    input_list.append(inps)
    aux_list.append(aux)
    output_list.append(output)

    #add p-neuron to your circuit
    p_layer.forward(circuit,[weight_1[i]],input_list[i][0],output_list[i],aux_list[i],input)

circuit.barrier()
circuit.draw('text',fold=300)


Now, we have a one-layer neural network. In the next step, we will on top
of this layer to add an output layer to the built quantum circuit.

As pointed by [QuantumFlow](https://www.nature.com/articles/s41467-020-20729-5),
the last layer can share the inputs. We will use *P_LYR_Circ* for the implementation,
where the first parameter shows how many neurons for the input, and the second
parameter shows how many neurons for the output.

**Note:** since the input of this layer is the output of the previous layer,
we do not need new input qubits. But, we will need an addition output qubits, which
can be obtained by using the *add_out_qubits* method.

In [ ]:
p_layer = P_LYR_Circ(2,2)

# Create output qubits
p_layer_output = p_layer.add_out_qubits(circuit)

# Build the second layer
p_layer.forward(circuit,weight_2,output_list,p_layer_output)

# Extract the results at the end of the quantum circuit
add_measure(circuit,p_layer_output,'reg')
print("Output layer created!")

circuit.draw('text',fold=300)

### simulation

In [ ]:
# result
qc_shots=8192
opt_counts = fire_ibmq(circuit,qc_shots,True)
(opt_mycount,bits) = analyze(opt_counts)
opt_class_prob=[]
for b in range(bits):
    opt_class_prob.append(float(opt_mycount[b])/qc_shots)
print("Simukation Result :",opt_class_prob)

### classical inference

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 =P_LYR(4, 2, bias=False)
        self.fc2 =P_LYR(2, 2, bias=False)
    def forward(self,x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

model = Net()

state_dict= model.state_dict()
state_dict["fc1.weight"] = weight_1
state_dict["fc2.weight"] = weight_2
model.load_state_dict(state_dict)

state = torch.tensor([[0,0,0,0]],dtype= torch.float)
output = model.forward(state)
print("classical inference Result :",output)